In [1]:
import rasterio
import geopandas as gpd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from shapely.geometry import mapping
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from rasterio.plot import reshape_as_raster, reshape_as_image

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import cross_val_score
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.windows import Window

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

import os
import csv
from rasterio.merge import merge
from collections import defaultdict
import pandas as pd


c:\Users\zhou.tang\software\anaconda3\envs\SOC_test\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\zhou.tang\AppData\Local\Temp\ipykernel_8848\1790777514.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should

In [2]:
def getPatchWithCsv(gps_csv, tif, patch_size, outfolder, month="July"):
    gps_csv = pd.read_csv(gps_csv)
    print(gps_csv.shape)

    gps_csv = gps_csv[gps_csv['Date'] == month].reset_index(drop=True)
    print(gps_csv.shape)
    lat_list = gps_csv['lat']
    lon_list = gps_csv['lon']
    folder_list = gps_csv['folder']
    class_list = gps_csv['class']
    print(lat_list)
    with rasterio.open(tif) as dataset:
        for i in range(len(lat_list)):
            p_lon = lon_list[i]
            p_lat = lat_list[i]
            fd = folder_list[i]
            cls = class_list[i]
            # clip
            py, px = dataset.index(p_lon, p_lat)
            window = rasterio.windows.Window(px - patch_size//2, py - patch_size//2, patch_size, patch_size)
            clip = dataset.read(window=window)
            # clip = clip/max_value
            clip = clip[0:6,:,:]
            # print(f"clip shape is {clip.shape}")
            has_nan = np.isnan(clip)
            
            if not np.any(has_nan):
                # meta data update
                transform = rasterio.windows.transform(window, dataset.transform)
                meta = dataset.meta.copy()
                meta.update({
                    'width': patch_size,
                    'height': patch_size,
                    'transform': transform,
                    'count': 6
                })

                # save clip
                out_tif = f"{fd}_{month}_{cls}_{str(p_lon)}_{str(p_lat)}_{patch_size}.tif"
                # clip_max = np.nanmax(clip)
                # clip = clip/clip_max
                
                out_tif = os.path.join(outfolder, out_tif)
                with rasterio.open(out_tif, 'w', **meta) as dst:
                    dst.write(clip)
            else:
                print(f"out tif name is {out_tif}")



In [39]:
gps_csv = "./data/all_data_two_date_gps_vi_savi_filter_0.9_split.csv"
tif = "./data/Modeling_data_7-26-2024/All_Paddock_26_JUL_2024_ortho_bgrent.tiff"
patch_size = 224
outfolder = "./data/patch"
month="July"
getPatchWithCsv(gps_csv=gps_csv, tif=tif,patch_size=patch_size,outfolder=outfolder,month=month)

(554, 17)
(355, 17)
0      30.877230
1      30.877097
2      30.876685
3      30.874522
4      30.876941
         ...    
350    30.876380
351    30.876701
352    30.876348
353    30.874737
354    30.876701
Name: lat, Length: 355, dtype: float64


In [40]:
gps_csv = "./data/all_data_two_date_gps_vi_savi_filter_0.9_split.csv"
tif = "./data/Modeling_data_09-22-2024/All_Paddock_22_SEP_2024_ortho_bgrent.tiff"
patch_size = 224
outfolder = "./data/patch"
month="Sep"
getPatchWithCsv(gps_csv=gps_csv, tif=tif,patch_size=patch_size,outfolder=outfolder,month=month)

(554, 17)
(199, 17)
0      30.874566
1      30.875349
2      30.874751
3      30.876414
4      30.876062
         ...    
194    30.875447
195    30.876806
196    30.875577
197    30.876953
198    30.875355
Name: lat, Length: 199, dtype: float64


In [5]:
gps_csv = "../data/all_data_two_date_gps_vi_savi_filter_0.9_split.csv"
tif = "../data/Modeling_data_09-22-2024/All_Paddock_22_SEP_2024_ortho_bgrent.tiff"
patch_size = 20
outfolder = "../data/patch_20"
month="Sep"
getPatchWithCsv(gps_csv=gps_csv, tif=tif,patch_size=patch_size,outfolder=outfolder,month=month)

(554, 17)
(199, 17)
0      30.874566
1      30.875349
2      30.874751
3      30.876414
4      30.876062
         ...    
194    30.875447
195    30.876806
196    30.875577
197    30.876953
198    30.875355
Name: lat, Length: 199, dtype: float64


In [7]:
gps_csv = "../data/all_data_two_date_gps_vi_savi_filter_0.9_split.csv"
tif = "../data/Modeling_data_7-26-2024/All_Paddock_26_JUL_2024_ortho_bgrent.tiff"
patch_size = 20
outfolder = "../data/patch_20"
month="July"
getPatchWithCsv(gps_csv=gps_csv, tif=tif,patch_size=patch_size,outfolder=outfolder,month=month)

(554, 17)
(355, 17)
0      30.877230
1      30.877097
2      30.876685
3      30.874522
4      30.876941
         ...    
350    30.876380
351    30.876701
352    30.876348
353    30.874737
354    30.876701
Name: lat, Length: 355, dtype: float64


In [8]:
gps_csv = "../data/all_data_two_date_gps_vi_savi_filter_0.9_split.csv"
tif = "../data/Modeling_data_7-26-2024/All_Paddock_26_JUL_2024_ortho_bgrent.tiff"
patch_size = 50
outfolder = "../data/patch_50"
month="July"
getPatchWithCsv(gps_csv=gps_csv, tif=tif,patch_size=patch_size,outfolder=outfolder,month=month)


tif = "../data/Modeling_data_09-22-2024/All_Paddock_22_SEP_2024_ortho_bgrent.tiff"
month="Sep"
getPatchWithCsv(gps_csv=gps_csv, tif=tif,patch_size=patch_size,outfolder=outfolder,month=month)

(554, 17)
(355, 17)
0      30.877230
1      30.877097
2      30.876685
3      30.874522
4      30.876941
         ...    
350    30.876380
351    30.876701
352    30.876348
353    30.874737
354    30.876701
Name: lat, Length: 355, dtype: float64
(554, 17)
(199, 17)
0      30.874566
1      30.875349
2      30.874751
3      30.876414
4      30.876062
         ...    
194    30.875447
195    30.876806
196    30.875577
197    30.876953
198    30.875355
Name: lat, Length: 199, dtype: float64
